# **DLT Pipeline**

In [0]:
import dlt
from pyspark.sql.functions import *

The Delta Live Tables (DLT) module is not supported on this cluster.
 You should either create a new pipeline or use an existing pipeline to run DLT code.

---------------------------------------------------------------------------
DLTImportException                        Traceback (most recent call last)
File <command-5919758866350932>, line 1
----> 1 import dlt
      2 from pyspark.sql.functions import *

File /databricks/python_shell/lib/dbruntime/PostImportHook.py:231, in _ImportHookChainedLoader.exec_module(self, module)
    229 def exec_module(self, module):
    230     try:
--> 231         self.loader.exec_module(module)
    232         notify_module_loaded(module)
    233     except (ImportError, AttributeError):

File /databricks/spark/python/dlt/__init__.py:22
     19 # DLT python module is not supported when Spark Connect is enabled. So, we throw an error
     20 # here.
     21 if is_remote_spark:
---> 22     raise DLTImportException(
     23         "Delta Live Tables module is not supported on Spark Connect clusters. "
     24     )
     26 from dlt.api import *
     27 from dlt.helpers import deprecation_warn

DLTImportExc

**Streaming Table - We create streaming table on a source (products_silver)**

In [0]:
#Expectations
my_rules = {
    "rule1" : "product_id IS NOT NULL",
    "rule2" : "product_name IS NOT NULL"
}

In [0]:
@dlt.table()

@dlt.expect_all_or_drop(my_rules)
def DimProducts_stage():
  df = spark.readStream.table("databricks_cata.silver.products_silver")
  return df

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-5919758866350928>, line 1
----> 1 @dlt.table(name = "DimProducts")
      2 
      3 def DimProducts_stage():
      4   df = spark.readStream.table("databricks_cata.products_silver")
      5   return df

NameError: name 'dlt' is not defined

**Streaming View**

In [0]:
@dlt.view

def DimProducts_view():
    df = spark.readStream.table("Live.DimProducts_stage")
    return df


**DimProducts**

In [0]:
dlt.create_streaming_table("DimProducts")

In [0]:
dlt.apply_changes(
    target = "DimProducts",
    source = "Live.DimProducts_view",
    keys = ["product_id"],
    sequence_by = "product_id",
    stored_as_scd_type = 2
)